# TPS-Aug-2022

In [1]:
class Config:
    NB = '002'

    raw_data_dir = '../data/raw/'
    processed_data_dir = '../data/processed/'
    interim_dir = '../data/interim/'
    submission_dir = '../data/submission/'

    random_seed = 42
    n_folds = 5

    row_id = 'id'
    target = 'failure'

## Import libralies

In [2]:
import gc
import warnings
warnings.filterwarnings('ignore')

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plotly_template = dict(
    layout=go.Layout(
        template='plotly_dark',
        font=dict(
            family="Franklin Gothic",
            size=12
        ),
        height=500,
        width=1000,
    )
)


color_palette = {
    'Bin': ['#016CC9','#E876A3'],
    'Cat5': ['#E876A3', '#E0A224', '#63B70D', '#6BCFF6', '#13399E'],
}

## Load and check data

### Load data

In [4]:
# Load data
##### Load train and Test set

df_train = pd.read_csv("../data/raw/train.csv")
df_test = pd.read_csv("../data/raw/test.csv")

In [5]:
df_train.describe()

,id,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
count,26570.000000,26320.000000,26570.000000,26570.000000,26570.000000,26570.000000,26570.000000,26189.000000,26032.000000,25894.000000,25774.000000,25633.000000,25522.000000,25343.000000,25270.000000,25102.000000,24969.000000,24796.000000,24696.000000,24561.000000,24460.000000,24286.000000,26570.000000
mean,13284.500000,127.826233,6.754046,7.240459,7.415883,8.232518,6.256568,17.791528,11.731988,17.127804,17.510759,11.716624,19.024714,11.430725,16.117711,19.172085,11.702464,15.652904,16.048444,14.995554,16.460727,701.269059,0.212608
std,7670.242662,39.030020,1.471852,1.456493,4.116690,4.199401,3.309109,1.001200,0.996085,0.996414,0.995980,1.000836,1.008591,0.999137,1.405978,1.520785,1.488838,1.155247,1.491923,1.549226,1.708935,123.304161,0.409160
min,0.000000,33.160000,5.000000,5.000000,0.000000,0.000000,0.000000,13.968000,8.008000,12.073000,12.715000,7.968000,15.217000,7.537000,9.323000,12.461000,5.167000,10.890000,9.140000,9.104000,9.701000,196.787000,0.000000
25%,6642.250000,99.987500,6.000000,6.000000,4.000000,5.000000,4.000000,17.117000,11.051000,16.443000,16.839000,11.045000,18.340250,10.757000,15.209000,18.170000,10.703000,14.890000,15.057000,13.957000,15.268000,618.961500,0.000000
50%,13284.500000,122.390000,6.000000,8.000000,7.000000,8.000000,6.000000,17.787000,11.733000,17.132000,17.516000,11.712000,19.021000,11.430000,16.127000,19.211500,11.717000,15.628500,16.040000,14.969000,16.436000,701.024500,0.000000
75%,19926.750000,149.152500,8.000000,8.000000,10.000000,11.000000,8.000000,18.469000,12.410000,17.805000,18.178000,12.391000,19.708000,12.102000,17.025000,20.207000,12.709000,16.374000,17.082000,16.018000,17.628000,784.090250,0.000000
max,26569.000000,385.860000,9.000000,9.000000,29.000000,29.000000,24.000000,21.499000,16.484000,21.425000,21.543000,15.419000,23.807000,15.412000,22.479000,25.640000,17.663000,22.713000,22.303000,21.626000,24.094000,1312.794000,1.000000


In [6]:
df_test.describe()

,id,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
count,20775.000000,20552.000000,20775.000000,20775.000000,20775.000000,20775.000000,20775.000000,20446.000000,20366.000000,20267.000000,20151.000000,20055.000000,19929.000000,19871.000000,19708.000000,19639.000000,19535.000000,19472.000000,19335.000000,19233.000000,19097.000000,19035.000000
mean,36957.000000,127.634895,7.733959,6.196823,7.453574,8.962407,6.126931,17.793466,11.727331,17.138080,17.515797,11.710919,19.030765,11.417921,16.123986,18.846571,11.914230,15.736104,16.123958,15.115915,16.636052,701.389816
std,5997.370257,39.154642,1.308535,1.917478,4.274477,4.334200,3.835881,1.002206,1.006834,1.008714,1.000067,1.001096,1.005401,0.999953,1.565414,1.588642,1.363253,1.357019,1.410569,1.545069,1.643463,130.205829
min,26570.000000,37.700000,6.000000,4.000000,0.000000,0.000000,0.000000,13.565000,7.384000,12.215000,13.539000,7.853000,14.885000,7.578000,9.167000,13.127000,6.116000,9.209000,8.415000,8.417000,10.162000,1.671000
25%,31763.500000,99.470000,6.000000,4.000000,4.000000,6.000000,3.000000,17.119000,11.048250,16.457000,16.847000,11.035000,18.351000,10.744000,15.095000,17.714000,11.069000,14.871000,15.238000,14.082000,15.512000,618.723500
50%,36957.000000,122.110000,7.000000,5.000000,7.000000,9.000000,6.000000,17.789000,11.729000,17.132000,17.510000,11.704000,19.040000,11.414000,16.109500,18.810000,11.941000,15.734000,16.119000,15.062000,16.706000,701.379000
75%,42150.500000,148.840000,9.000000,7.000000,10.000000,12.000000,8.000000,18.478000,12.411000,17.824500,18.197000,12.385000,19.707000,12.093000,17.156000,19.967500,12.791000,16.605000,17.001500,16.107000,17.781000,784.872500
max,47344.000000,385.570000,9.000000,9.000000,30.000000,33.000000,28.000000,21.389000,15.623000,21.681000,21.183000,15.828000,23.092000,15.091000,23.354000,24.950000,18.962000,21.677000,23.140000,22.097000,22.270000,1242.786000


In [7]:
features_list = [col for col in df_train.columns if col not in [Config.row_id, Config.target, 'product_code']]
features_list

['loading',
 'attribute_0',
 'attribute_1',
 'attribute_2',
 'attribute_3',
 'measurement_0',
 'measurement_1',
 'measurement_2',
 'measurement_3',
 'measurement_4',
 'measurement_5',
 'measurement_6',
 'measurement_7',
 'measurement_8',
 'measurement_9',
 'measurement_10',
 'measurement_11',
 'measurement_12',
 'measurement_13',
 'measurement_14',
 'measurement_15',
 'measurement_16',
 'measurement_17']

## 基本情報

In [8]:
df_train.groupby(['product_code','attribute_0', 'attribute_1', 'attribute_2', 'attribute_3']).count()

,,,,,id,loading,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
product_code,attribute_0,attribute_1,attribute_2,attribute_3,,,,,,,,,,,,,,,,,,,,,
A,material_7,material_8,9,5,5100,5051,5100,5100,5100,5032,4998,4949,4925,4924,4913,4875,4843,4804,4774,4770,4784,4723,4712,4674,5100
B,material_5,material_5,8,8,5250,5209,5250,5250,5250,5166,5155,5135,5117,5057,5038,4999,4983,4963,4952,4889,4909,4862,4791,4800,5250
C,material_7,material_8,5,8,5765,5716,5765,5765,5765,5689,5642,5603,5583,5573,5523,5495,5484,5469,5432,5404,5337,5327,5323,5296,5765
D,material_7,material_5,6,6,5112,5063,5112,5112,5112,5043,5016,5001,4956,4941,4917,4874,4871,4824,4798,4733,4720,4706,4710,4646,5112
E,material_7,material_6,6,9,5343,5281,5343,5343,5343,5259,5221,5206,5193,5138,5131,5100,5089,5042,5013,5000,4946,4943,4924,4870,5343


In [9]:
df_test.groupby(['product_code','attribute_0', 'attribute_1', 'attribute_2', 'attribute_3']).count()

,,,,,id,loading,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
product_code,attribute_0,attribute_1,attribute_2,attribute_3,,,,,,,,,,,,,,,,,,,,
F,material_5,material_6,6,4,5422,5355,5422,5422,5422,5345,5312,5301,5258,5246,5201,5186,5183,5137,5081,5093,5042,5014,4966,4959
G,material_5,material_6,9,7,5107,5058,5107,5107,5107,5019,4990,4981,4945,4941,4889,4898,4815,4800,4795,4798,4766,4747,4669,4677
H,material_7,material_7,7,9,5018,4967,5018,5018,5018,4933,4930,4881,4868,4823,4830,4786,4749,4741,4708,4694,4664,4631,4615,4585
I,material_7,material_5,9,5,5228,5172,5228,5228,5228,5149,5134,5104,5080,5045,5009,5001,4961,4961,4951,4887,4863,4841,4847,4814


## 各特徴量とTargetの関係

In [13]:
float_cols = [f for f in df_test.columns if df_test[f].dtype in [float, int]]
bin_num = 20
df_target_prob = pd.DataFrame(data={'idx':range(bin_num-1)})

fig = make_subplots(
    rows=6,
    cols=4,
    #start_cell='bottom-left', # どのセルを起点とするか
    subplot_titles=float_cols,
    shared_xaxes=False, # x軸を共有する場合
    shared_yaxes=False, # y軸を共有する場合
)

for idx, f in enumerate(float_cols):
    min_val = min(df_train[f].min(), df_test[f].min())
    max_val = max(df_train[f].max(), df_test[f].max())
    bins = np.linspace(min_val, max_val, bin_num)

    total, _ = np.histogram(df_train[f], bins=bins)
    failures, _ = np.histogram(df_train[f][df_train.failure == 1], bins=bins)

    df_target_prob[f'{f}_fail'] = failures
    df_target_prob[f'{f}_total'] = total
    df_target_prob[f'{f}_prob'] = failures / total


    #with warnings.catch_warnings(): # ignore divide by zero for empty bins
    #    warnings.filterwarnings('ignore', category=RuntimeWarning)

    fig.add_trace(
        go.Scatter(
            x=(bins[1:] + bins[:-1]) / 2,
            y=failures / total,
            #y=total,
            name='Failure probability',
            mode='markers',
            #marker=dict(size=1, color='red'),
            #line=dict(color='black'),
            customdata=total,
            hovertemplate="<b>%{y:.2f}</b> <br>total: %{customdata:.2f}<extra></extra>",
        ),
        row=int(idx/4) + 1,
        col=idx%4 + 1
    )

fig.update_layout(
    barmode='overlay',
    width=1500,
    height=1500,
)

fig.show()

In [14]:
# 相関行列を生成
df_corr = df_train.corr()

fig = go.Figure(layout=plotly_template['layout'])

fig.add_trace(
    go.Heatmap(
        x=df_corr.columns,
        y=df_corr.index,
        z=np.array(df_corr),
        #annotation_text=np.around(np.array(df_corr), decimals=2),
        #y=total,
        name='Corr',
        #mode='markers',
        #marker=dict(size=1, color='red'),
        #line=dict(color='black'),
        #customdata=total,
        #hovertemplate="<b>%{y:.2f}</b> <br>total: %{customdata:.2f}<extra></extra>",
    ),
)

fig.update_layout(
    width=1500,
    height=1500,
)

fig.show()

### Check distribution

In [ ]:
fig = go.Figure(layout=plotly_template['layout'])

fig.add_trace(
    go.Histogram(
        x=df_train['measurement_4'],
        name='Train',
        histnorm='probability',
        #marker=dict(size=15, color='red'),
        #line=dict(color='black')
    )
)
fig.add_trace(
    go.Histogram(
        x=df_test['measurement_4'],
        name='Test',
        histnorm='probability',
        #marker=dict(size=15, color='red'),
        #line=dict(color='black')
    )
)

fig.update_layout(barmode='overlay')
fig.show()

### Train と Test　の分布

In [ ]:
fig = make_subplots(
    rows=6,
    cols=4,
    #start_cell='bottom-left', # どのセルを起点とするか
    subplot_titles=features_list,
    shared_xaxes=False, # x軸を共有する場合
    shared_yaxes=False, # y軸を共有する場合
)

for idx, col in enumerate(features_list):
    fig.add_trace(
        go.Histogram(
            x=df_train[col],
            name=f'{col}(Train)',
            histnorm='probability',
            marker=dict(color=color_palette['Bin'][0]),
            #line=dict(color='black')
        ),
        row=int(idx/4) + 1,
        col=idx%4 + 1
    )
    fig.add_trace(
        go.Histogram(
            x=df_test[col],
            name=f'{col}(Test)',
            histnorm='probability',
            marker=dict(color=color_palette['Bin'][1]),
            opacity=0.5
            #line=dict(color='black')
        ),
        row=int(idx/4) + 1,
        col=idx%4 + 1
    )


fig.update_layout(
    barmode='overlay',
    width=1500,
    height=1500,
)
fig.show()

### Targetの分布

In [ ]:
df_train[Config.target].unique()

In [ ]:
fig = make_subplots(
    rows=6,
    cols=4,
    #start_cell='bottom-left', # どのセルを起点とするか
    subplot_titles=features_list,
    shared_xaxes=False, # x軸を共有する場合
    shared_yaxes=False, # y軸を共有する場合
)

for idx, col in enumerate(features_list):
    fig.add_trace(
        go.Histogram(
            x=df_train[df_train[Config.target] == 0][col],
            name=f'{col}(False)',
            histnorm='probability',
            marker=dict(color=color_palette['Bin'][0]),
            #line=dict(color='black')
        ),
        row=int(idx/4) + 1,
        col=idx%4 + 1
    )
    fig.add_trace(
        go.Histogram(
            x=df_train[df_train[Config.target] == 1][col],
            name=f'{col}(True)',
            histnorm='probability',
            marker=dict(color=color_palette['Bin'][1]),
            opacity=0.5
            #line=dict(color='black')
        ),
        row=int(idx/4) + 1,
        col=idx%4 + 1
    )


fig.update_layout(
    barmode='overlay',
    width=1500,
    height=1500,
)
fig.show()

In [ ]:
fig = make_subplots(
    rows=6,
    cols=4,
    #start_cell='bottom-left', # どのセルを起点とするか
    subplot_titles=features_list,
    shared_xaxes=False, # x軸を共有する場合
    shared_yaxes=False, # y軸を共有する場合
)

for idx, col in enumerate(features_list):
    fig.add_trace(
        go.Box(
            x=df_train[df_train[Config.target] == 0][col],
            name=f'{col}(False)',
            #histnorm='probability',
            marker=dict(color=color_palette['Bin'][0]),
            #line=dict(color='black')
        ),
        row=int(idx/4) + 1,
        col=idx%4 + 1
    )
    fig.add_trace(
        go.Box(
            x=df_train[df_train[Config.target] == 1][col],
            name=f'{col}(True)',
            #histnorm='probability',
            marker=dict(color=color_palette['Bin'][1]),
            #opacity=0.5
            #line=dict(color='black')
        ),
        row=int(idx/4) + 1,
        col=idx%4 + 1
    )


fig.update_layout(
    barmode='overlay',
    width=1500,
    height=1500,
)
fig.show()

### product_code ごとの分布

In [ ]:
fig = make_subplots(
    rows=6,
    cols=4,
    #start_cell='bottom-left', # どのセルを起点とするか
    subplot_titles=features_list,
    shared_xaxes=False, # x軸を共有する場合
    shared_yaxes=False, # y軸を共有する場合
)

for idx, col in enumerate(features_list):
    fig.add_trace(
        go.Histogram(
            x=df_train[(df_train['product_code'] == 'A') & (df_train[Config.target] == 0)][col],
            name=f'{col}(False)',
            histnorm='probability',
            marker=dict(color=color_palette['Cat5'][0]),
            #line=dict(color='black')
        ),
        row=int(idx/4) + 1,
        col=idx%4 + 1
    )
    fig.add_trace(
        go.Histogram(
            x=df_train[(df_train['product_code'] == 'A') & (df_train[Config.target] == 1)][col],
            name=f'{col}(True)',
            histnorm='probability',
            marker=dict(color=color_palette['Cat5'][1]),
            opacity=0.5
            #line=dict(color='black')
        ),
        row=int(idx/4) + 1,
        col=idx%4 + 1
    )


fig.update_layout(
    barmode='overlay',
    width=1500,
    height=1500,
)
fig.show()

In [ ]:
df_target_pivot = pd.DataFrame(data={'Positive':df_train.groupby('product_code')[Config.target].mean() * 100})
df_target_pivot['Negative'] = (1 - df_train.groupby('product_code')[Config.target].mean()) * 100
df_target_pivot

In [ ]:
fig = go.Figure(layout=plotly_template['layout'])

fig.add_trace(go.Bar(x=df_target_pivot.index, y=df_target_pivot['Positive'], name='Positive',
                     text=df_target_pivot['Positive'], texttemplate='%{text:.0f}%',
                     textposition='inside', insidetextanchor="middle",
                     marker=dict(color=color_palette['Bin'][1]),
                     hovertemplate = "<b>%{x}</b><br>Paid accounts: %{y:.2f}%"))

fig.add_trace(go.Bar(x=df_target_pivot.index, y=df_target_pivot['Negative'], name='Negative',
                     text=df_target_pivot['Negative'], texttemplate='%{text:.0f}%',
                     textposition='inside',insidetextanchor="middle",
                     marker=dict(color=color_palette['Bin'][0]),
                     hovertemplate = "<b>%{x}</b><br>Default accounts: %{y:.2f}%"))

fig.update_layout(title='Distribution of Failure',
                  barmode='relative', yaxis_ticksuffix='%', width=1400,
                  #legend=dict(orientation="h", traceorder="reversed", yanchor="bottom", y=0, xanchor="left", x=0),
                  legend=dict(orientation="h", traceorder="reversed", y=1.1, x=0.8),
                  xaxis_title='product code', yaxis_title='Percentage of Positive')

fig.show()

## 欠損値

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_train[df_train['product_code'] == 'A']['attribute_0'].unique()

In [ ]:
df_train

## 欠損を各プロダクトの中央値で補完する

In [ ]:
f = lambda x: x.fillna(x.median())
df_train = df_train.groupby('product_code').transform(f)
df_test = df_test.groupby('product_code').transform(f)

## loadingの検証

In [ ]:
df_train.groupby([Config.target, 'product_code']).mean()